# Analyse des marnages de réservoirs EPANET

Un notebook réalisé par [@Valentin Mounier](https://github.com/valmoun "Github de l'auteur") pour calculer les marnages min et max de réservoirs d'un réseau EPANET.

## Importations

### Modèle EPANET

In [1]:
# Importations EPANET
import ganessa.en2emu as sim_et
from ganessa.en2emu import *

	Epanet version is: 2.2 (from owa-epanet 2.2.4)


### Interface graphique

In [2]:
# Importations de l'interface utilisateur
import os
from ipywidgets import Text
from IPython.display import display

## Paramétrage simulation EPANET

### Sélection modèle

In [3]:
# Widget de saisie
text_chooser = Text(
    value='../data/modele.inp',
    placeholder='Entrer le chemin complet du fichier .inp',
    description='Fichier .inp :',
    disabled=False,
    continuous_update=False
)

# Affichage du widget
display(text_chooser)

Text(value='../data/modele.inp', continuous_update=False, description='Fichier .inp :', placeholder='Entrer le…

### Paramètres de la simulation

In [4]:
# Widget de saisie de la durée de simulation
text_duration = Text(
    value='24',
    placeholder='Entrer la durée de simulation',
    description='Durée (h) :',
    disabled=False,
    continuous_update=False
)

# Affichage du widget
display(text_duration)

Text(value='24', continuous_update=False, description='Durée (h) :', placeholder='Entrer la durée de simulatio…

### Vérifications inputs utilisateur

In [5]:
# Stockage des inputs dans des variables
model_file = text_chooser.value
duration = int(text_duration.value)

# Créer les fichiers de sortie .rpt et .out
rpt_file = model_file.replace('.inp', '.rpt')
out_file = model_file.replace('.inp', '.out')

# Vérification de l'existence et de la validité du fichier .inp
if os.path.isfile(model_file):
    print(f"Le fichier {model_file} existe.")
    # Vérification de l'extension .inp
    if model_file.endswith('.inp'):
        print(f"Le fichier {model_file} est un fichier .inp valide.")
    else:
        print(f"Le fichier {model_file} n'est pas un fichier .inp valide.")
else:
    print(f"Le fichier {model_file} n'existe pas.")

# Vérification durée de simulation
if duration <= 0:
    print(f"Durée de simulation invalide : {duration} heures")
else:
    print(f"Durée de simulation valide : {duration} heures")

Le fichier ../data/modele.inp existe.
Le fichier ../data/modele.inp est un fichier .inp valide.
Durée de simulation valide : 24 heures


## Initialisation du modèle EPANET

In [6]:
# Initialisation du modèle EPANET
try:
    model = ENmodel(model_file, None)
    print("Model initialized successfully.")
except Exception as e:
    print(f"Error initializing model: {e}")

Default epanet project created: 0x1b4366a1890
Model initialized successfully.


### Récupération des ID de réservoirs

In [7]:
# Récupération de la liste des tanks
try:
    print("Retrieving tank IDs...")
    model.update_indexes()  # Update tank and pipe indices
    tank_ids = [et.ENgetnodeid(ix) for ix in model.tank_index]  # Get tank IDs
    print("Tank IDs retrieved successfully.")
    
    # Display tank IDs
    for tank_id in tank_ids:
        print(f"Tank ID: {tank_id}")
        
except Exception as e:
    print(f"Error during tank ID retrieval: {e}")

Retrieving tank IDs...
Tank IDs retrieved successfully.
Tank ID: 26


## Lancement de la simulation

In [8]:
# Extraction des pressions
print("Extracting tank pressures...")
try:
    _, _, pressures, _ = model.runH_results_part([], tank_ids, tank_ids)
    tanks = {tank_id: pressures[tank_id] for tank_id in tank_ids}
    print("Tank pressure extraction complete.")
except Exception as e:
    print(f"Error during pressure extraction: {e}")

Extracting tank pressures...
Model has 40 links and 36 nodes.
Running simulation over 198000 s and collecting results
 links=0 nodes=1 tanks=1
	  0% - t= 0
	 10% - t= 21600
	 20% - t= 39600
	 30% - t= 61200
	 40% - t= 79200
	 50% - t= 100800
	 60% - t= 118800
	 70% - t= 140400
	 80% - t= 158400
	 90% - t= 180000
	100% - t= 198000
Tank pressure extraction complete.


## Calculs des marnages des réservoirs sur 24h

In [9]:
# Calcul des marnages
stats = {}
for tank_id, pressures in tanks.items():
    if pressures:
        stats[tank_id] = {
            "max_pressure": max(pressures),
            "min_pressure": min(pressures),
        }
    else:
        stats[tank_id] = {
            "max_pressure": None,
            "min_pressure": None,
        }

# Affichage des stats finales

In [10]:
print("\n=== Tank Pressure Statistics ===")

# Affichage des statistiques
analysis_file_path = os.path.join(os.path.dirname(model_file), "analysis.txt")
with open(analysis_file_path, "w") as analysis_file:
    for tank_id, stat in stats.items():
        line = f"Tank {tank_id}: Max = {stat['max_pressure']}, Min = {stat['min_pressure']}\n"
        print(line.strip())
        analysis_file.write(line)

print(f"\nAnalysis saved to {analysis_file_path}")

# Fermeture du modèle
if 'model' in locals() and model:
    model.close()

# Suppression du fichier rapport .rpt
if os.path.isfile(rpt_file):
    os.remove(rpt_file)
    print(f"Deleted report file: {rpt_file}")


=== Tank Pressure Statistics ===
Tank 26: Max = 19.841366117285673, Min = 17.1229377503588

Analysis saved to ../data\analysis.txt
Default project 0x1b4366a1890 is being deleted
Deleted report file: ../data/modele.rpt
